1. PARSE XML FILES
2. TURN INTO TREE
3. IF FAILS, PARSE AS MANY OF THE ENTRIES AS POSSIBLE

In [12]:
import traceback
import xml.etree.ElementTree as ET
from pathlib import Path
import re
import shutil
import os

import git

repo = git.Repo('.', search_parent_directories=True)
os.chdir(repo.working_tree_dir)

srcpaths=(
    "first_run_xmls",
)

xmls = []
for d in srcpaths:
    l = list(Path(d).rglob("*.xml"))
    l = [p for p in l if ".repair." not in p.name]
    xmls += l
print(len(xmls), "XML files found,", len(set([f.name for f in xmls])), "unique. duplicates =", [f for f in xmls if [g.name for g in xmls].count(f.name) > 1])

185 XML files found, 185 unique. duplicates = []


In [13]:
dstdir=Path("first_run_xmls-lxml")

In [14]:
if dstdir.exists():
    shutil.rmtree(dstdir)
dstdir.mkdir(exist_ok=True)

In [16]:
import tqdm as tqdm
from lxml import etree as ET2

successes_1 = []
failed_1 = []
with tqdm.tqdm(xmls) as pbar:
    for fpath in pbar:
        try:
            tree = ET2.parse(fpath)
            successes_1.append(fpath)
            tree.write(dstdir/(fpath.name), pretty_print=True)
        except ET2.XMLSyntaxError as ex:
            failed_1.append(fpath)
        except ET.ParseError as ex:
            failed_1.append(fpath)
        pbar.set_postfix({
            "success": len(successes_1),
            "failed": len(failed_1),
        })

print("ROUND 1:", len(failed_1), "files failed parsing")
(dstdir/"1_repair_success_1.txt").write_text("\n".join(map(str, successes_1)))

 10%|▉         | 18/185 [05:14<1:00:21, 21.69s/it, success=2, failed=16]

In [ ]:

from lxml import etree as ET2
parser = ET2.XMLParser(recover=True)

successes_2 = []
failed_2 = []
with tqdm.tqdm(failed_1) as pbar:
    for fpath in pbar:
        try:
            tree = ET2.parse(fpath, parser=parser)
            successes_2.append(fpath)
            tree.write(dstdir/(fpath.name + ".repair.xml"), pretty_print=True)
        except ET2.XMLSyntaxError as ex:
            failed_2.append(fpath)
        pbar.set_postfix({
            "success": len(successes_2),
            "failed": len(failed_2),
        })

print("ROUND 2:", len(failed_2), "files failed parsing")
(dstdir/"1_repair_success_2.txt").write_text("\n".join(map(str, successes_2)))